## This is the code for testing "multiple_linear_regression"

### y = w1 * x1 + w2 * x2 + w3 * x3 + ... + b

In [82]:
### import packages 
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

data = pd.read_csv('./mlr/Salary_Data2.csv')

In [83]:
### label encoding: e.g., degree: 0, 1, 2 ...
data["EducationLevel"] = data["EducationLevel"].map({"高中以下": 0, "大學": 1, "碩士以上": 2})

In [84]:
### one hot encoding

onehot_encoder = OneHotEncoder()
onehot_encoder.fit(data[["City"]])

city_encoded = onehot_encoder.transform(data[["City"]]).toarray()

data[["CityA", "CityB", "CityC"]] = city_encoded
data = data.drop(["City", "CityC"], axis = 1)


//anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:395: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  check_array(X, dtype=np.int)
//anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:110: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, che

In [85]:
from sklearn.model_selection import train_test_split

x = data[["YearsExperience", "EducationLevel", "CityA", "CityB"]]
y = data["Salary"]

# separate training & testing sets: 20% used for testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=86) 
# random_state means do not change the data selected, if change, use other numbers ... 

x_train = x_train.to_numpy()
#print (len(x_train))

### Salary = w1 * YearsExperience + w2 * EducationLevel + w3 * CityA + w4 * CityB + b

In [100]:
import numpy as np


w = np.array([1, 2, 3, 4])
b = 1.0

y_pred = (x_train*w).sum(axis = 1) + b # sum of each row

In [101]:
cost = ((y_train - y_pred)**2).mean()

def compute_cost(x, y, w, b):
    """
    This funct is to calculate cost function
    """
    y_pred = (x*w).sum(axis = 1) + b
    cost = ((y - y_pred)**2).mean()
    
    return cost

In [102]:
compute_cost(x_train, y_train, w, b)

1818.4746428571423

### w1 - w1_grad * learning_rate
### w2 - w2_grad * learning_rate

In [103]:
### cost = (y - y_pred)**2 = (y - (w1*x1+w2*x2+w3*x3+w4*x4+b)**2
y_pred = (x_train*w).sum(axis = 1) + b 
b_grad = (y_pred - y_train).mean()


In [104]:
def compute_grad(x, y, w, b):
    """
    This funct used to calculate gradients
    """
    y_pred = (x*w).sum(axis=1) + b 
    ll = x.shape[1]; w_grad = np.zeros(ll)
    b_grad = (y_pred - y).mean()
    for i in range(ll):
        w_grad[i] = (x[:,i]*(y_pred - y)).mean()
    
    return w_grad, b_grad

In [105]:
w = np.array([1, 2, 3, 4])
b = 1.0

compute_grad(x_train, y_train, w, b)

(array([-2.19e+02, -5.82e+01, -1.58e+01, -4.99e+00]), -39.78214285714285)

In [108]:
w = np.array([1, 2, 2, 4])
b = 1.0
learning_rate = 0.001
w_grad, b_grad = compute_grad(x_train, y_train, w, b)

# old cost
print (compute_cost(x_train, y_train, w, b))
    
w = w - w_grad*learning_rate
b = b - b_grad*learning_rate

print (compute_cost(x_train, y_train, w, b))

1850.703214285714
1744.3700355532972


In [116]:
np.set_printoptions(formatter={'float': '{: .2e}'.format})

def grad_descent(x, y, w_init, b_init, l_rate, cost_funct, grad_funct, run_inter, p_inter = 1000):
    """
    This funct used to calculate the gradient descent with input changes
    """
    c_hist = []; w_hist = []; b_hist = []
    
    w = w_init
    b = b_init
    
    for i in range(run_inter):
        w_gradient, b_gradient = compute_grad(x, y, w, b)

        w = w - w_gradient * learning_rate
        b = b - b_gradient * learning_rate
        cost = compute_cost(x,y,w,b)
        
        c_hist.append(cost)
        w_hist.append(w)
        b_hist.append(b)
        
        if i%p_inter == 0:
            print (f"Ieration {i:5}: Cost {cost: .5e}, w: {w}, b: {b}")
    
    return w, b, c_hist, w_hist, b_hist

In [118]:
### Now, use the function

w_init = np.array([1, 2, 2, 4])
b_init = 0
l_rate = 2.0e-3
run_inter = 20000
w_final, b_final, c_hist, w_hist, b_hist = grad_descent(x_train, y_train, w_init, b_init, l_rate, compute_cost, compute_grad, run_inter, p_inter = 1000)

Ieration     0: Cost  1.82101e+03, w: [ 1.23e+00  2.06e+00  2.02e+00  4.01e+00], b: 0.041317857142857135
Ieration  1000: Cost  1.32320e+02, w: [ 4.63e+00  1.26e+01  3.98e+00  2.44e+00], b: 5.703698438557687
Ieration  2000: Cost  6.47313e+01, w: [ 3.27e+00  1.72e+01  4.80e+00  1.12e+00], b: 8.468380003492305
Ieration  3000: Cost  4.53617e+01, w: [ 2.57e+00  1.93e+01  5.15e+00  5.60e-02], b: 10.166839940194103
Ieration  4000: Cost  3.88260e+01, w: [ 2.22e+00  2.04e+01  5.20e+00 -8.39e-01], b: 11.281130634626818
Ieration  5000: Cost  3.58606e+01, w: [ 2.05e+00  2.08e+01  5.08e+00 -1.62e+00], b: 12.069866805155327
Ieration  6000: Cost  3.39947e+01, w: [ 1.96e+00  2.09e+01  4.87e+00 -2.32e+00], b: 12.672785725294537
Ieration  7000: Cost  3.25480e+01, w: [ 1.92e+00  2.09e+01  4.60e+00 -2.95e+00], b: 13.16600374498556
Ieration  8000: Cost  3.13210e+01, w: [ 1.90e+00  2.09e+01  4.30e+00 -3.54e+00], b: 13.59138114176089
Ieration  9000: Cost  3.02462e+01, w: [ 1.90e+00  2.08e+01  4.00e+00 -4.08e

In [119]:
w_final, b_final

(array([ 1.95e+00,  1.98e+01,  1.28e+00, -8.11e+00]), 16.923929443026303)

In [120]:
y_pred = (w_final*x_test).sum(axis=1) + b_final

pd.DataFrame({
    "y_pred": y_pred,
    "y_test": y_test
})

,y_pred,y_test
19,28.357258,31.6
35,46.393795,36.7
16,70.480844,72.7
22,67.914612,63.6
7,71.261555,70.3
24,52.721647,48.3
11,45.613083,48.3
1,71.734073,80.5


In [121]:
compute_cost(x_test, y_test, w_final, b_final)

29.070264027069683

## Now, use Feature Scaling to increase the speed of gradient descent 

### x1 ~ (1, 10) while x2, x3 and x4 ~ (0, 2)

### method: standardization = (x - mean(x)) / sdt

In [124]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x_train) # only use training data, no testing data!!

x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)


array([[ 5.43e-02, -1.47e+00,  9.31e-01, -3.46e-01],
       [ 1.25e+00, -2.22e-01, -1.07e+00, -3.46e-01],
       [-1.03e+00, -1.47e+00,  9.31e-01, -3.46e-01],
       [-1.81e-02, -1.47e+00,  9.31e-01, -3.46e-01],
       [ 1.18e+00, -2.22e-01, -1.07e+00, -3.46e-01],
       [-3.44e-01, -2.22e-01,  9.31e-01, -3.46e-01],
       [-1.43e+00,  1.02e+00, -1.07e+00, -3.46e-01],
       [-1.10e+00, -1.47e+00,  9.31e-01, -3.46e-01],
       [ 1.03e+00, -2.22e-01, -1.07e+00, -3.46e-01],
       [-7.78e-01, -2.22e-01,  9.31e-01, -3.46e-01],
       [-1.99e-01, -2.22e-01,  9.31e-01, -3.46e-01],
       [ 8.87e-01,  1.02e+00, -1.07e+00, -3.46e-01],
       [ 1.76e+00,  1.02e+00, -1.07e+00,  2.89e+00],
       [-1.32e+00,  1.02e+00, -1.07e+00, -3.46e-01],
       [-5.61e-01, -2.22e-01,  9.31e-01, -3.46e-01],
       [ 8.14e-01,  1.02e+00, -1.07e+00, -3.46e-01],
       [-1.21e+00,  1.02e+00, -1.07e+00, -3.46e-01],
       [ 1.76e+00,  1.02e+00, -1.07e+00,  2.89e+00],
       [-1.27e-01, -2.22e-01,  9.31e-01, -3.46

In [125]:
w_final, b_final, c_hist, w_hist, b_hist = grad_descent(x_train, y_train, w_init, b_init, l_rate, compute_cost, compute_grad, run_inter, p_inter = 1000)

Ieration     0: Cost  2.77242e+03, w: [ 1.01e+00  2.01e+00  1.99e+00  4.00e+00], b: 0.050325
Ieration  1000: Cost  3.96064e+02, w: [ 2.71e+00  9.05e+00 -2.66e+00  2.05e+00], b: 31.839232020652823
Ieration  2000: Cost  7.90508e+01, w: [ 3.04e+00  1.13e+01 -3.19e+00 -9.37e-02], b: 43.52786769061649
Ieration  3000: Cost  3.27807e+01, w: [ 3.46e+00  1.24e+01 -3.01e+00 -1.37e+00], b: 47.82572554827677
Ieration  4000: Cost  2.48038e+01, w: [ 3.89e+00  1.31e+01 -2.70e+00 -2.13e+00], b: 49.40602821885431
Ieration  5000: Cost  2.28339e+01, w: [ 4.24e+00  1.35e+01 -2.40e+00 -2.59e+00], b: 49.987098280578984
Ieration  6000: Cost  2.20953e+01, w: [ 4.52e+00  1.38e+01 -2.14e+00 -2.89e+00], b: 50.20075508374654
Ieration  7000: Cost  2.17423e+01, w: [ 4.72e+00  1.40e+01 -1.94e+00 -3.09e+00], b: 50.27931571274255
Ieration  8000: Cost  2.15580e+01, w: [ 4.87e+00  1.42e+01 -1.78e+00 -3.23e+00], b: 50.308202096591536
Ieration  9000: Cost  2.14589e+01, w: [ 4.98e+00  1.43e+01 -1.65e+00 -3.33e+00], b: 50.3